<a href="https://colab.research.google.com/github/CarlosMejia07/Proyecto-Kaggle-UdeA-20252/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importe, descompresión e inspección de los datos**

In [27]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:    296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


**Importe de las librerias a usar y lectura del archivo train**

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

z = pd.read_csv("train.csv")
print ("Estructura de los datos cargados", z.shape)

Estructura de los datos cargados (692500, 21)


**Se dejan de lado las columnas que no aporten a la predicción, preliminarmente**

In [29]:
z = z.drop(columns=['ID', 'PERIODO_ACADEMICO'])

**Detección y reemplazo de valores nulos**

In [30]:
# Separar valores numéricos y no numéricos
numeric_cols = z.select_dtypes(include=np.number).columns
non_numeric_cols = z.select_dtypes(exclude=np.number).columns

# Reemplazar valores nulos de columnas numéricas con la media
z[numeric_cols] = z[numeric_cols].fillna(z[numeric_cols].median())

# Reemplazar valores nulos de columnas no numéricas por la moda
for col in non_numeric_cols:
    z[col] = z[col].fillna(z[col].mode()[0])

z.isna().sum()

,0
E_PRGM_ACADEMICO,0
E_PRGM_DEPARTAMENTO,0
E_VALORMATRICULAUNIVERSIDAD,0
E_HORASSEMANATRABAJA,0
F_ESTRATOVIVIENDA,0
F_TIENEINTERNET,0
F_EDUCACIONPADRE,0
F_TIENELAVADORA,0
F_TIENEAUTOMOVIL,0
E_PRIVADO_LIBERTAD,0


**Codificación de las variables con valores nominales y ordinales**

In [34]:
z.F_ESTRATOVIVIENDA.value_counts()

,count
F_ESTRATOVIVIENDA,
Estrato 2,264808
Estrato 3,210685
Estrato 1,111991
Estrato 4,65514
Estrato 5,23608
Estrato 6,12605
Sin Estrato,3289


In [31]:
# Reemplazar valores si & no por 0 & 1
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
z.replace({'Si': 1, 'No': 0, 'N': 0}, inplace=True)

# Reemplazar valores de variables ordinales
z['F_EDUCACIONPADRE'] = z['F_EDUCACIONPADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')
z['F_EDUCACIONMADRE'] = z['F_EDUCACIONMADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')

orden_educacion = [
    'Desconocido',
    'Ninguno',
    'Primaria incompleta',
    'Primaria completa',
    'Secundaria (Bachillerato) incompleta',
    'Secundaria (Bachillerato) completa',
    'Técnica o tecnológica incompleta',
    'Técnica o tecnológica completa',
    'Educación profesional incompleta',
    'Educación profesional completa',
    'Postgrado'
]
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[orden_educacion])
z['F_EDUCACIONPADRE'] = encoder.fit_transform(z[['F_EDUCACIONPADRE']])
z['F_EDUCACIONMADRE'] = encoder.fit_transform(z[['F_EDUCACIONMADRE']])


In [33]:
orden_matrícula = [
    'No pagó matrícula',
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones'
]
encoder = OrdinalEncoder(categories=[orden_matrícula])
z['E_VALORMATRICULAUNIVERSIDAD'] = encoder.fit_transform(z[['E_VALORMATRICULAUNIVERSIDAD']])

In [35]:
orden_estrato = [
    'Sin Estrato',
    'Estrato 1',
    'Estrato 2',
    'Estrato 3',
    'Estrato 4',
    'Estrato 5',
    'Estrato 6'
]
encoder = OrdinalEncoder(categories=[orden_estrato])
z['F_ESTRATOVIVIENDA'] = encoder.fit_transform(z[['F_ESTRATOVIVIENDA']])